In [58]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

- https://towardsdatascience.com/the-4-recommendation-engines-that-can-predict-your-movie-tastes-109dc4e10c52m

In [23]:
movies = pd.read_csv('../data/netflix-rotten-tomatoes-metacritic-imdb.csv')
korea_netflix = pd.read_csv('../data/favorite_contents_unified_modified.csv')

In [20]:
movies = movies[(~movies['Genre'].isna())]

In [15]:
movies

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,...,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon",...,2021-03-04,NaN,https://www.netflix.com/watch/81305978,https://www.imdb.com/title/tt8945942,"Trapped in a frozen car during a blizzard, a p...",1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81336456,https://www.imdb.com/title/tt6132758,"Upon moving into a new place, a 20-something r...",1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15467,Brand New Day,"Documentary, Music","Australian Comedies,Romantic Comedies,Australi...",English,Movie,8.6,Australia,1-2 hour,Amos Gitai,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/70125340,NaN,In an idyllic port town on Australias west coa...,14.0,http://cdn1.nflximg.net/images/5543/12045543.jpg,NaN,NaN,NaN
15473,Daniel Arends: Blessuretijd,Comedy,"Stand-up Comedy,International Movies,Comedies",Dutch,Movie,8.8,Belgium,1-2 hour,Doesjka van Hoogdalem,Daniël Arends,...,2015-04-14,NaN,https://www.netflix.com/watch/70281233,NaN,"In his third show, Daniël Arends argues that g...",174.0,http://occ-0-768-769.1.nflxso.net/dnm/api/v6/e...,NaN,NaN,NaN
15477,DreamWorks Happy Holidays from Madagascar,"Animation, Comedy, Family","TV Comedies,Kids TV,Animal Tales,TV Cartoons,T...",English,Series,8.4,"Belgium,Switzerland,United States,Germany,Unit...",< 30 minutes,NaN,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/70259784,NaN,Madagascar goes wild with holiday spirit in th...,71.0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,NaN,NaN,NaN
15478,DreamWorks Holiday Classics,"Animation, Comedy, Family","TV Comedies,Kids TV,TV Cartoons,TV Programmes,...",English,Series,8.2,"Belgium,Switzerland,Germany,Argentina,United K...",< 30 minutes,NaN,NaN,...,2015-04-14,Fox,https://www.netflix.com/watch/70221348,NaN,Join your DreamWorks friends for these four ho...,82.0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN


In [12]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['Genre'])

In [14]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [17]:
# Build a 1-dimensional array with movie titles
titles = movies['Title']
indices = pd.Series(movies.index, index=movies['Title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [18]:
genre_recommendations('Moxie').head(20)

1083       If Anything Happens I Love You
3764              Sitara: Let Girls Dream
3268                            Control Z
97       Ginny & Georgia - The Afterparty
125                              Sisyphus
549                           Before Love
617                               Applaus
1498                                Fugue
1575                         The Deadline
2243                      Ginny & Georgia
3922                                Fenix
4741                        Mimi and Lisa
5609                                 Draw
7991                     The Bulbuls Nest
8577          Pocoyo and The Space Circus
11005                        Four Minutes
13864                LeapFrog: Numberland
13866              LeapFrog: Phonics Farm
14                        To Kill a Child
83                   Communication & Lies
Name: Title, dtype: object

In [46]:
movies.head(5)

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,...,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon",...,2021-03-04,NaN,https://www.netflix.com/watch/81305978,https://www.imdb.com/title/tt8945942,"Trapped in a frozen car during a blizzard, a p...",1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81336456,https://www.imdb.com/title/tt6132758,"Upon moving into a new place, a 20-something r...",1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN


In [112]:
movie_korea = movies.copy()


In [113]:
movie_korea.head(1)

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN


In [114]:
movie_korea = movie_korea[~movie_korea['Country Availability'].isna()]

In [115]:
ca_before_rg = movie_korea['Country Availability']
reg_ca = []

for i in range(len(ca_before_rg)):
    try:
        target = movie_korea.iloc[i,6].replace(' ','').lower().split(',')
        reg_ca.append([x for x in target])

#         print(target)
    except:
        reg_ca.append('')

In [116]:
reg_ca

[['thailand'],
 ['canada'],
 ['canada'],
 ['belgium', 'netherlands'],
 ['lithuania',
  'poland',
  'france',
  'iceland',
  'italy',
  'spain',
  'greece',
  'czechrepublic',
  'belgium',
  'portugal',
  'canada',
  'hungary',
  'mexico',
  'slovakia',
  'sweden',
  'southafrica',
  'netherlands',
  'germany',
  'thailand',
  'turkey',
  'singapore',
  'romania',
  'argentina',
  'israel',
  'switzerland',
  'australia',
  'unitedkingdom',
  'brazil',
  'malaysia',
  'india',
  'colombia',
  'hongkong',
  'japan',
  'southkorea',
  'unitedstates',
  'russia'],
 ['thailand'],
 ['poland'],
 ['poland'],
 ['lithuania',
  'poland',
  'france',
  'italy',
  'spain',
  'greece',
  'belgium',
  'portugal',
  'netherlands',
  'germany',
  'switzerland',
  'unitedkingdom',
  'iceland',
  'czechrepublic'],
 ['lithuania',
  'poland',
  'france',
  'italy',
  'spain',
  'greece',
  'czechrepublic',
  'belgium',
  'portugal',
  'hungary',
  'slovakia',
  'netherlands',
  'germany',
  'romania',
  's

In [117]:
target = movie_korea.iloc[3,6]

In [118]:
text = re.sub('[^a-zA-Z]',', ',target).strip()
# text = text.split(',')

text

'Belgium, Netherlands'

In [119]:
def str_to_list(str_):
    try:
        text = re.sub('[^a-zA-Z,]','',str_).strip()
        text = text.split(',')
        return text
    except:
        return None

In [120]:
movie_korea['Country Availability'] = movie_korea['Country Availability'].apply(str_to_list)

In [121]:
movie_korea.head()

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,[Thailand],< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,[Canada],1-2 hour,Coky Giedroyc,Caitlin Moran,...,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,[Canada],1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon",...,2021-03-04,NaN,https://www.netflix.com/watch/81305978,https://www.imdb.com/title/tt8945942,"Trapped in a frozen car during a blizzard, a p...",1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"[Belgium, Netherlands]",< 30 minutes,NaN,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81336456,https://www.imdb.com/title/tt6132758,"Upon moving into a new place, a 20-something r...",1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"[Lithuania, Poland, France, Iceland, Italy, Sp...",1-2 hour,Stephen Irwin,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN


In [122]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

In [123]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [124]:
values = to_1D(movie_korea['Country Availability']).value_counts().keys().tolist()
counts = to_1D(movie_korea['Country Availability']).value_counts().tolist()
unique_items = dict(zip(values, counts))
unique_items

{'UnitedKingdom': 6311,
 'CzechRepublic': 6237,
 'Japan': 6115,
 'Hungary': 5989,
 'Canada': 5930,
 'Singapore': 5876,
 'Thailand': 5876,
 'India': 5826,
 'Slovakia': 5820,
 'Australia': 5820,
 'Romania': 5759,
 'UnitedStates': 5718,
 'SouthAfrica': 5553,
 'Lithuania': 5541,
 'Germany': 5539,
 'Russia': 5491,
 'Switzerland': 5472,
 'Belgium': 5437,
 'Malaysia': 5394,
 'Iceland': 5364,
 'Greece': 5324,
 'Israel': 5198,
 'France': 5194,
 'HongKong': 5188,
 'Mexico': 5148,
 'Italy': 5141,
 'Argentina': 5138,
 'Poland': 5136,
 'Brazil': 5057,
 'Netherlands': 5021,
 'Spain': 5010,
 'Colombia': 4867,
 'SouthKorea': 4845,
 'Turkey': 4814,
 'Sweden': 4751,
 'Portugal': 4685}

In [125]:
ca_bool_df = boolean_df(movie_korea['Country Availability'], unique_items)

In [126]:
ca_int_df = ca_bool_df.astype(int)

In [130]:
movie_korea.insert(6, 'southkorea', ca_int_df['SouthKorea'])

In [136]:
movie_korea_yes_df = movie_korea[movie_korea['southkorea'] == 1]
movie_korea_yes_df

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,southkorea,Country Availability,Runtime,Director,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,1,"[Lithuania, Poland, France, Iceland, Italy, Sp...",1-2 hour,Stephen Irwin,...,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN
27,Comrades: Almost a Love Story,"Drama, Romance","Romantic Dramas,Chinese Movies,Dramas,Romantic...","Cantonese, Mandarin, English",Movie,6.7,1,[SouthKorea],1-2 hour,Peter Ho-Sun Chan,...,2021-03-01,NaN,https://www.netflix.com/watch/60020365,https://www.imdb.com/title/tt0117905,Two young Chinese mainlanders transplanted to ...,5047.0,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BZjg5MT...,NaN,NaN
28,The Closet,Comedy,"Korean Movies,Horror Movies,Mysteries",French,Movie,3.8,1,[SouthKorea],1-2 hour,Francis Veber,...,2021-03-01,"Miramax, Gaumont",https://www.netflix.com/watch/81405032,https://www.imdb.com/title/tt0243493,A recent widowers move into a new house takes ...,16657.0,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BMGRmNz...,https://www.youtube.com/watch?v=ou9YG0fUztA,YouTube
53,The Bold Type,Drama,"TV Comedies,TV Dramas,US TV Shows",English,Series,4.1,1,"[Lithuania, Poland, Italy, Spain, Greece, Czec...",< 30 minutes,NaN,...,2021-03-01,NaN,https://www.netflix.com/watch/80176085,https://www.imdb.com/title/tt6116060,"At a womens magazine in New York, three millen...",9767.0,https://occ-0-768-769.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BZDkyNj...,NaN,NaN
55,Lobster Cop,"Comedy, Crime","Dark Comedies,Action & Adventure,Chinese Movie...",Mandarin,Movie,7.5,1,[SouthKorea],1-2 hour,Xinyun Li,...,2021-02-28,NaN,https://www.netflix.com/watch/81007862,https://www.imdb.com/title/tt8489090,"To nab a drug kingpin, four cops open a seafoo...",143.0,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BMDQ3ND...,https://www.youtube.com/watch?v=Otq5XzH5p8Y,YouTube
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,K-POP Extreme Survival,NaN,"TV Dramas,TV Programmes,TV Comedies,Romantic T...",NaN,Series,NaN,1,"[SouthKorea, Argentina, UnitedKingdom, Austral...",< 30 minutes,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/80031595,NaN,Seung Yeon decides to chase her dream of becom...,NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN
15476,DreamWorks Shreks Swamp Stories,NaN,"Animal Tales,Family Comedies,Family Adventures...",NaN,Series,NaN,1,"[Russia, HongKong, Hungary, Australia, SouthKo...",< 30 minutes,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/70242630,NaN,"Shrek celebrates Halloween, Puss in Boots is c...",NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN
15477,DreamWorks Happy Holidays from Madagascar,"Animation, Comedy, Family","TV Comedies,Kids TV,Animal Tales,TV Cartoons,T...",English,Series,8.4,1,"[Belgium, Switzerland, UnitedStates, Germany, ...",< 30 minutes,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/70259784,NaN,Madagascar goes wild with holiday spirit in th...,71.0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,NaN,NaN,NaN
15478,DreamWorks Holiday Classics,"Animation, Comedy, Family","TV Comedies,Kids TV,TV Cartoons,TV Programmes,...",English,Series,8.2,1,"[Belgium, Switzerland, Germany, Argentina, Uni...",< 30 minutes,NaN,...,2015-04-14,Fox,https://www.netflix.com/watch/70221348,NaN,Join your DreamWorks friends for these four ho...,82.0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN


In [ ]:
movie_korea[movie_korea]

In [12]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movie_korea_yes_df['Genre'])

In [14]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [137]:
# Build a 1-dimensional array with movie titles
titles = movies['Title']
indices = pd.Series(movie_korea_yes_df.index, index=movie_korea_yes_df['Title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [141]:
genre_recommendations('Lobster Cop').head(20)

55                               Lobster Cop
178                           O Filho Eterno
270                                     50M2
354                                  Sputnik
659                                   Feride
924                        Crazy Crying Lady
1649                              Photograph
3105                             Cruel Peter
3262                             The Weekend
3369                            No Surrender
3417                     My Moochy Boyfriend
3562                     The Ghost Who Walks
4042                                    1945
4054                           Crazy Romance
4591    Escape from Mr. Lemoncello’s Library
4867                              South Park
4921                            The Outsider
4945       Merry Men: The Real Yoruba Demons
5232                                   A Day
5524                 Wanda Sykes: Not Normal
Name: Title, dtype: object